<a href="https://colab.research.google.com/github/halen48/DCGAN/blob/main/GAN_Celeb_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><a href="https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html">Deep Convolutional Generative Adversarial Networks</a></h1>
<h2>Autor do código: <a href="https://twitter.com/fchollet">François Chollet</a>

<h2>Adaptação e transposição para o Notebook: <a href="https://github.com/halen48/">Guilherme Novaes</a></h2>

#Dependências

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os
import gdown
from zipfile import ZipFile


#Constantes e Parâmetros

In [2]:
batch_size =  256#@param {type:"integer"}
image_size = 64 #@param {type:"integer"}
latent_dim = 128 #@param {type:"integer"}
image_channels = 3 #@param {type:"integer"}
lr = 0.0001 #@param {type:"number"}
epochs = 200 #@param {type:"number"}
num_imgs_save =  9#@param {type:"number"}

work_path = '/drive/MyDrive/GAN/' #@param{type:"string"}

import os
from google.colab import drive
import re

drive.mount('/content/drive')

dir = os.getcwd()+work_path

import sys
sys.path.insert(0, dir)

work_path = '.' + work_path
try:
  os.makedirs(work_path)
except FileExistsError:
  print("Diretório [%s] já criado..."%work_path)
for i in range(num_imgs_save):
  try:
    os.makedirs(work_path+"/%d"%i)
  except FileExistsError:
    print("Diretório [%d] já criado..."%i)

Mounted at /content/drive
Diretório [./drive/MyDrive/GAN/] já criado...
Diretório [0] já criado...
Diretório [1] já criado...
Diretório [2] já criado...
Diretório [3] já criado...
Diretório [4] já criado...
Diretório [5] já criado...
Diretório [6] já criado...
Diretório [7] já criado...
Diretório [8] já criado...


#Carregar o dataset <a href="http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html">Celeb-A Faces dataset</a>

In [ ]:
try:
  os.makedirs("celeba_gan")

  url = "https://drive.google.com/uc?id=1O7m1010EJjLE5QxLZiM9Fpjs7Oj6e684"
  output = "celeba_gan/data.zip"
  gdown.download(url, output, quiet=True)
  with ZipFile("celeba_gan/data.zip", "r") as zipobj:
    zipobj.extractall("celeba_gan")
except FileExistsError:
  print("Dataset já foi baixado!")

dataset = keras.preprocessing.image_dataset_from_directory(
    "celeba_gan", label_mode=None, image_size=(image_size, image_size), batch_size=batch_size
)

In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(15, 15))

i = 0
plot_dim = 7
for x in dataset:
  # define subplot
  plt.subplot(plot_dim, plot_dim, 1 + i)
  # turn off axis
  plt.axis('off')
  # plot raw pixel data
  plt.imshow((x.numpy()).astype("int32")[0])
  i+=1
  if(i == plot_dim*plot_dim):
    break
plt.show()

##Normaliza os valores das imagens

In [ ]:
dataset = dataset.map(lambda x: x / 255.0)

#Definições dos modelos

##Discriminador

In [ ]:
try:
  discriminator = keras.models.load_model(work_path+'discriminator')
except OSError:
  print("Criando novo discriminator")
  discriminator = keras.Sequential(
    [
        keras.Input(shape=(image_size, image_size, image_channels)),
        layers.Conv2D(64, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Flatten(),
        layers.Dropout(0.2),
        layers.Dense(1, activation="sigmoid"),
    ],
    name="discriminator",
)
discriminator.summary()


##Gerador

In [ ]:
try:
  generator = keras.models.load_model(work_path+'generator')
except OSError:
  print("Criando novo generator")
  generator = keras.Sequential(
      [
          keras.Input(shape=(latent_dim,)),
          layers.Dense(8 * 8 * 128),
          layers.Reshape((8, 8, 128)),
          layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"),
          layers.LeakyReLU(alpha=0.2),
          layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding="same"),
          layers.LeakyReLU(alpha=0.2),
          layers.Conv2DTranspose(512, kernel_size=4, strides=2, padding="same"),
          layers.LeakyReLU(alpha=0.2),
          layers.Conv2D(3, kernel_size=5, padding="same", activation="sigmoid"),
      ],
      name="generator",
  )
generator.summary()


##Rotina para salvar as imagens geradas durante as iterações

In [ ]:
tf.config.run_functions_eagerly(True)

def save_imgs(epoch, num_img, latent_dim, generator, discriminator):
  generator.save(work_path+"generator")
  discriminator.save(work_path+"discriminator")
  random_latent_vectors = tf.random.normal(shape=(num_img, latent_dim))
  generated_images = generator(random_latent_vectors)
  generated_images *= 255
  generated_images.numpy()
  for id in range(num_img):
      img = keras.preprocessing.image.array_to_img(generated_images[id])
      img.save(work_path+"%d/generated_img_%03d.png" % (id,epoch))

In [ ]:
class GANMonitor(keras.callbacks.Callback):
    def __init__(self, num_img=num_imgs_save, latent_dim=latent_dim):
        self.num_img = num_imgs_save
        self.latent_dim = latent_dim

    def on_epoch_end(self, epoch, logs=None):
        save_imgs(epoch,self.num_img, self.latent_dim, self.model.generator, self.model.discriminator)        

##Rotina de treinamento da DCGAN

In [ ]:
class GAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super(GAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(GAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn
        self.d_loss_metric = keras.metrics.Mean(name="d_loss")
        self.g_loss_metric = keras.metrics.Mean(name="g_loss")

    @property
    def metrics(self):
        return [self.d_loss_metric, self.g_loss_metric]

    def train_step(self, real_images):
        # Sample random points in the latent space
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Decode them to fake images
        generated_images = self.generator(random_latent_vectors)

        # Combine them with real images
        combined_images = tf.concat([generated_images, real_images], axis=0)

        # Assemble labels discriminating real from fake images
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )
        # Add random noise to the labels - important trick!
        labels += 0.05 * tf.random.uniform(tf.shape(labels))

        # Train the discriminator
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        # Sample random points in the latent space
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Assemble labels that say "all real images"
        misleading_labels = tf.zeros((batch_size, 1))

        # Train the generator (note that we should *not* update the weights
        # of the discriminator)!
        with tf.GradientTape() as tape:
            predictions = self.discriminator(self.generator(random_latent_vectors))
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        # Update metrics
        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)


        return {
            "d_loss": self.d_loss_metric.result(),
            "g_loss": self.g_loss_metric.result(),
        }


#Treinamento

##Rotina de treinamento

In [ ]:
gan = GAN(discriminator=discriminator, generator=generator, latent_dim=latent_dim)
gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=lr),
    g_optimizer=keras.optimizers.Adam(learning_rate=lr),
    loss_fn=keras.losses.BinaryCrossentropy(),
)

history = gan.fit(
    dataset, epochs=epochs, callbacks=[GANMonitor(num_img=num_imgs_save, latent_dim=latent_dim)]
)


##Histórico do treinamento da GAN

In [ ]:
plt.plot(history.history['g_loss'])
plt.plot(history.history['d_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Generator', 'Discriminator'], loc='upper left')
plt.show()

#Resultados

##Exemplo de nova imagem

In [ ]:
from PIL import Image
for id in range(num_imgs_save):
  frames = [Image.open(work_path+"%d/generated_img_%03d.png"%(id,epoch)) for epoch in range(epochs)]
  frames[0].save(work_path+"pessoa_%d.gif"%id,save_all = True, append_images=frames[1:], duration=1/len(frames), loop=0)
    